### [Analysis] Change rate decomposition: ${\delta}$Mort${^{(2)}}$

#### This set of demonstrative code shows how to estimate the ambient O${_3}$ exposure-associated mortality using curved relative risk model. Here the curved exposure-response associations are defined as the accumulative risks (not unit incremental risks), that values estimated from meta-regression directly represent the absolute additional risks. 

#### For 1990-2019 change rate decomposition, here is the last key feature that all input parameters are changed into 2019.  

In [1]:
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

#### Ambient O${_3}$ dataset is developed in global scale with 1/8°×1/8° spatial resolution, and thus we have to select the grids in China territory. 

In [2]:
data = pd.read_csv('ozone_90_mort_est_total(1119).csv')
data = data[data['arcgis_ID'] == 44]

#### We first should import the exposure-response (concentration-risk) curve-shaped lists, and here we also show what such dataset should look like. 

In [3]:
cr_copd = pd.read_csv('copd_CR_curve_values.csv')
cr_copd['lnRR'] = np.log(cr_copd['RR'])
cr_copd['lnSD'] = (np.log(cr_copd['RR'])-np.log(cr_copd['RR_lb']))/1.96

cr_cvds = pd.read_csv('cvds_CR_curve_values.csv')
cr_cvds['lnRR'] = np.log(cr_cvds['RR'])
cr_cvds['lnSD'] = (np.log(cr_cvds['RR'])-np.log(cr_cvds['RR_lb']))/1.96

#### For the sake of quick computation (e.g. less than 10 minutes on JASMIN supercomputation system) as a demonstration, we have simplified some trivial aspects: (1) we cluster the age ≥25 population rather than 5-yr stratification. (2) we do not calculate the arithmetic mean and standard deviations, as we choose to report the median with 95% uncertainty intervals. 

In [5]:
start = time.process_time()

for t in range(0,61607,1):
    
    ## Extract the age ≥25 risked proportion 
    pop_all_age       = data.iloc[t,25]
    pop_all_age_sigma = data.iloc[t,26]
    pop_25_age        = data.iloc[t,27]
    pop_25_age_sigma  = data.iloc[t,28]
    
    ## Extract urban and rural population and MDA8h-ozone exposure 
    pop_urban_grid = data.iloc[t,10]
    o3_mda8h_urban = data.iloc[t,22] 
    pop_rural_grid = data.iloc[t,11]
    o3_mda8h_rural = data.iloc[t,23] 
    
    ## Generate Gaussian distributed samples 
    pop_all_age_dist = np.random.normal(pop_all_age, pop_all_age_sigma, 1000) 
    pop_25_age_dist = np.random.normal(pop_25_age, pop_25_age_sigma, 1000)

    urban_seq = min(999, int(np.round(o3_mda8h_urban, 1)*10))
    rural_seq = min(999, int(np.round(o3_mda8h_rural, 1)*10))
    
    if cr_copd.iloc[urban_seq-1,2] < 1:
        lnRR_copd_urban = 0
        lnRR_sigma_copd_urban = 0 
    else:
        lnRR_copd_urban = cr_copd.iloc[urban_seq-1,4]
        lnRR_sigma_copd_urban = cr_copd.iloc[urban_seq-1,5]

    if cr_copd.iloc[rural_seq-1,2] < 1:
        lnRR_copd_rural = 0
        lnRR_sigma_copd_rural = 0
    else:
        lnRR_copd_rural = cr_copd.iloc[rural_seq-1,4]
        lnRR_sigma_copd_rural = cr_copd.iloc[rural_seq-1,5]

    if cr_cvds.iloc[urban_seq-1,2] < 1:
        lnRR_cvds_urban = 0
        lnRR_sigma_cvds_urban = 0 
    else:
        lnRR_cvds_urban = cr_cvds.iloc[urban_seq-1,4]
        lnRR_sigma_cvds_urban = cr_cvds.iloc[urban_seq-1,5]

    if cr_cvds.iloc[rural_seq-1,2] < 1:
        lnRR_cvds_rural = 0
        lnRR_sigma_cvds_rural = 0
    else:
        lnRR_cvds_rural = cr_cvds.iloc[rural_seq-1,4]
        lnRR_sigma_cvds_rural = cr_cvds.iloc[rural_seq-1,5]

    RR_dist_copd_urban = np.random.normal(lnRR_copd_urban, lnRR_sigma_copd_urban, 1000)
    RR_dist_copd_rural = np.random.normal(lnRR_copd_rural, lnRR_sigma_copd_rural, 1000)    

    RR_dist_cvds_urban = np.random.normal(lnRR_cvds_urban, lnRR_sigma_cvds_urban, 1000)
    RR_dist_cvds_rural = np.random.normal(lnRR_cvds_rural, lnRR_sigma_cvds_rural, 1000)  

    ## Extract the baseline mortality rate 
    copd_25_mort           = data.iloc[t,37]
    copd_25_mort_sigma     = data.iloc[t,38]
    cvds_25_mort           = data.iloc[t,41]
    cvds_25_mort_sigma     = data.iloc[t,42]

    copd_25_mort_dist = np.random.normal(copd_25_mort, copd_25_mort_sigma, 1000)
    cvds_25_mort_dist = np.random.normal(cvds_25_mort, cvds_25_mort_sigma, 1000)

    ## Start the Monte Carlo bootstrap simulation realisations 
    delta_mort_copd_urban = copd_25_mort_dist * (1 - np.exp(-RR_dist_copd_urban)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    delta_mort_copd_rural = copd_25_mort_dist * (1 - np.exp(-RR_dist_copd_rural)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 

    delta_mort_cvds_urban = cvds_25_mort_dist * (1 - np.exp(-RR_dist_cvds_urban)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    delta_mort_cvds_rural = cvds_25_mort_dist * (1 - np.exp(-RR_dist_cvds_rural)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    
    ## Summarise the results 
    est_copd_urban = np.nanpercentile(delta_mort_copd_urban,50) 
    lb_copd_urban  = np.nanpercentile(delta_mort_copd_urban,2.5)
    ub_copd_urban  = np.nanpercentile(delta_mort_copd_urban,97.5)

    est_copd_rural = np.nanpercentile(delta_mort_copd_rural,50) 
    lb_copd_rural  = np.nanpercentile(delta_mort_copd_rural,2.5)
    ub_copd_rural  = np.nanpercentile(delta_mort_copd_rural,97.5)
    
    est_cvd_urban = np.nanpercentile(delta_mort_cvds_urban,50) 
    lb_cvd_urban  = np.nanpercentile(delta_mort_cvds_urban,2.5)
    ub_cvd_urban  = np.nanpercentile(delta_mort_cvds_urban,97.5)

    est_cvd_rural = np.nanpercentile(delta_mort_cvds_rural,50) 
    lb_cvd_rural  = np.nanpercentile(delta_mort_cvds_rural,2.5)
    ub_cvd_rural  = np.nanpercentile(delta_mort_cvds_rural,97.5)

    ## Store the distribution statistics into the DataFrame     
    data.iloc[t,73]  = est_copd_urban
    data.iloc[t,74]  =  lb_copd_urban
    data.iloc[t,75]  =  ub_copd_urban

    data.iloc[t,78]  = est_copd_rural
    data.iloc[t,79]  =  lb_copd_rural
    data.iloc[t,80]  =  ub_copd_rural
    
    data.iloc[t,83]  = est_cvd_urban
    data.iloc[t,84]  =  lb_cvd_urban
    data.iloc[t,85]  =  ub_cvd_urban

    data.iloc[t,88]  = est_cvd_rural
    data.iloc[t,89]  =  lb_cvd_rural
    data.iloc[t,90]  =  ub_cvd_rural
    
    if t%1000 == 0:
        print(t) ## Add a tracer to check the simulation progresses 
    
elapsed = (time.process_time() - start)
print('Time used:', elapsed, "s")    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
Time used: 230.15253048800002 s


#### Now let's have a look at the demonstrative estimations. Here we use the BNNDv2 for population ambient O${_3}$ exposure assessment, other than the fusion of BNNDv2, CEML and TAP, and thus the simulation results might be slightly deviant from the conclusions in our main analysis. 

In [6]:
copd_urban_est = np.round(data['copd_25_death_urban_est'].sum()/1000,1)
copd_urban_ub = np.round(data['copd_25_death_urban_ub'].sum()/1000,1)
copd_urban_lb = np.round(data['copd_25_death_urban_lb'].sum()/1000,1)
copd_rural_est = np.round(data['copd_25_death_rural_est'].sum()/1000,1)
copd_rural_ub = np.round(data['copd_25_death_rural_ub'].sum()/1000,1)
copd_rural_lb = np.round(data['copd_25_death_rural_lb'].sum()/1000,1)

print("Urban population mortality of COPD:  "+str(copd_urban_est)+" ["+str(copd_urban_lb)+" to "+str(copd_urban_ub)+"] thousands")
print("Rural population mortality of COPD:  "+str(copd_rural_est)+" ["+str(copd_rural_lb)+" to "+str(copd_rural_ub)+"] thousands")

Urban population mortality of COPD:  40.4 [29.4 to 53.1] thousands
Rural population mortality of COPD:  123.7 [92.0 to 159.8] thousands


In [7]:
cvds_urban_est = np.round(data['cvd_25_death_urban_est'].sum()/1000,1)
cvds_urban_ub = np.round(data['cvd_25_death_urban_ub'].sum()/1000,1)
cvds_urban_lb = np.round(data['cvd_25_death_urban_lb'].sum()/1000,1)
cvds_rural_est = np.round(data['cvd_25_death_rural_est'].sum()/1000,1)
cvds_rural_ub = np.round(data['cvd_25_death_rural_ub'].sum()/1000,1)
cvds_rural_lb = np.round(data['cvd_25_death_rural_lb'].sum()/1000,1)

print("Urban population mortality of CVDs:  "+str(cvds_urban_est)+" ["+str(cvds_urban_lb)+" to "+str(cvds_urban_ub)+"] thousands")
print("Rural population mortality of CVDs:  "+str(cvds_rural_est)+" ["+str(cvds_rural_lb)+" to "+str(cvds_rural_ub)+"] thousands")

Urban population mortality of CVDs:  17.0 [11.0 to 23.3] thousands
Rural population mortality of CVDs:  75.0 [53.4 to 98.1] thousands


In [8]:
data.to_csv('change rate decomposition delta_mort_2019.csv', index=False)